In [1]:
import math
import time
import os
import cv2 
import keyboard
import pickle
import numpy as np
from PIL import Image, ImageOps, ImageGrab
from matplotlib import pyplot as plt
from skimage import io, color, measure, morphology
from scipy import ndimage
from scipy.ndimage import rotate
import copy
%matplotlib inline

In [87]:
# image = cv2.imread(r'training\3.png')[75:835, 395:1426]
# At the end is crop to monitor

# Item detection
Scrap items appear to all appear at roughly the same shape, size, and colour. This suggests template matching as a solution. However, upon examining the training images, it was noted that each scrap item had the exact same colour albeit for a small difference observed in one case. This suggests that precise colour thresholding will be very effective at isolating the scrap items from the rest of the image. The steps for the item detection are as follows:

- RGB colour threeshold applied to the image to identify scrap items.
- Dilate segments of scrap items. This helps prevent map elements that sometimes break up the appearance of scrap as appearing as more than one scrap item when it is only one.

In [88]:
monitor = cv2.imread(r'training\3.png')[75:835, 395:1426]

# Colour threshold for scrap items
scrap_lower_bound = (1, 254, 172)
scrap_upper_bound = (2, 255, 177)

# Colour threshold the image
color_mask = cv2.inRange(monitor, scrap_lower_bound, scrap_upper_bound)

# Dilate segments
kernel_size = (5, 5)
kernel = np.ones(kernel_size, np.uint8)
item_mask = cv2.dilate(color_mask, kernel, iterations=3)

cv2.imshow('Original Image', image)
cv2.imshow('Binary Mask', item_mask)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Enemy detection
Similar to scrap items, enemies all have the same shape and colour, but have differences in size. There are also issues with other objects appearing the same colour as the enemies on the monitor. This necessitates a requirement for more sophisticated methods of detection. Circular Hough Transforms were tried, but too difficult to implement. A simple solution was found by using morphological opening to filter out non-circular shaped objects that proved better than the circular hough transform. This will be used to improve the enemy detection method.

- RGB colour threshold applied to the image to filter out noise and any objects not red.
- Morphological opening (erode, then dilate) to remove non-circular segments. 

In [89]:
monitor = cv2.imread(r'training\17.png')[75:835, 395:1426]
# 5, 8

# Colour threshold for enemies
enemy_lower_bound = (1, 1, 85)
enemy_upper_bound = (25, 25, 255)

# Colour threshold the image
color_mask = cv2.inRange(monitor, enemy_lower_bound, enemy_upper_bound)

# Morphological Opening
kernel_size = (4, 4)
kernel = np.ones(kernel_size, np.uint8)
dilated_mask = cv2.erode(color_mask, kernel, iterations=2)
enemy_mask = cv2.dilate(dilated_mask, kernel, iterations=3)

cv2.imshow('Original Image', monitor)
cv2.imshow('Binary Mask', enemy_mask)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Player Angle Detection

Simplify the method used to detect what direction the player is looking in, and return one of four simple directions: (front, behind, left, right). There were somoe difficulties here as the colour thresholding is not as effective. However, the cases where this become an issue are quite limited. The process for detecting the angle the player is looking in is as follows:

- Crop the image to just the player.
- Blue colour thresholding on the image to isolate the player.
- Average a centroid for the segmented areas and calculate the angle based on the centre of the image to that point.

In [6]:
def calculate_angle(binary_mask):
    # Calculate the centroid of the white pixels in the binary mask
    Y, X = np.where(binary_mask == 255)
    centroid_x = np.mean(X)
    centroid_y = np.mean(Y)

    # Determine the center point of the image
    image_center_x, image_center_y = binary_mask.shape[1] / 2, binary_mask.shape[0] / 2

    # Calculate the angle to the centroid of the largest segment
    delta_x = centroid_x - image_center_x
    delta_y = image_center_y - centroid_y  # Image coordinates are top-left, so we invert the y-axis
    angle_rad = math.atan2(delta_y, delta_x)  # Calculate angle in radians
    # Adjust the angle so 0 degrees is 'up' and positive angles are clockwise
    angle_deg = (90 - math.degrees(angle_rad)) % 360

    return angle_deg

monitor = cv2.imread(r'training\16.png')[75:835, 395:1426]
player = monitor[307:473, 453:619]

# Colour thresholds for the player
player_lower_bound = (90, 1, 1)
player_upper_bound = (255, 200, 10)

# Colour threshold the image
color_mask = cv2.inRange(player, player_lower_bound, player_upper_bound)

# Dilate to fill in any noise
kernel_size = (5, 5)
kernel = np.ones(kernel_size, np.uint8)
player_mask = cv2.dilate(color_mask, kernel, iterations=3)

angle = calculate_angle(player_mask)
print(angle)

cv2.imshow('Monitor', player_mask)
cv2.imshow('Player', player)

cv2.waitKey(0)
cv2.destroyAllWindows()

45.14058270723698


# Supporting Functions
Support functions such as converting a degree angle to a direction.

In [4]:
def get_direction_from_angle(angle):
    # Normalize the angle to the range [0, 360)
    angle = angle % 360

    # Determine the direction based on the angle range
    if (angle >= 315 and angle < 360) or (angle >= 0 and angle < 45):
        return 'front'
    elif 45 <= angle < 135:
        return 'right'
    elif 135 <= angle < 225:
        return 'behind'
    else:  # 225 <= angle < 315
        return 'left'

In [ ]:
def get_segment_coords(image_mask):
    contours, _ = cv2.findContours(image_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    

# Testing All Together
Now we 